In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import os
import time

mp_face = mp.solutions.face_mesh.FaceMesh()
mp_hand = mp.solutions.hands.Hands(max_num_hands=1)

def collect_label_data(label, mode, num_frames=200):
    save_dir = f"dataset/{mode}"
    os.makedirs(save_dir, exist_ok=True)

    cap = cv2.VideoCapture(0)
    collected = 0
    data = []
    collecting = False

    print(f"\nPrepare to collect '{label}' data for {mode}. Press 'c' to start collecting {num_frames} frames.")
    print("Press 'q' to quit anytime.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to read frame from camera.")
            break

        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        face_result = mp_face.process(rgb)
        hand_result = mp_hand.process(rgb)

        detected = False
        landmarks = []

        if mode == 'face' and face_result.multi_face_landmarks:
            face_landmarks = face_result.multi_face_landmarks[0].landmark
            landmarks = [coord for lm in face_landmarks for coord in (lm.x, lm.y)]
            detected = True

        elif mode == 'hand' and hand_result.multi_hand_landmarks:
            hand_landmarks = hand_result.multi_hand_landmarks[0].landmark
            landmarks = [coord for lm in hand_landmarks for coord in (lm.x, lm.y)]
            detected = True

        cv2.putText(frame, f"Mode: {mode} | Label: {label}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        if collecting:
            if detected:
                data.append(landmarks)
                collected += 1
                cv2.putText(frame, f"Collecting... {collected}/{num_frames}",
                            (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, f"No {mode} detected - skipping frame",
                            (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            if collected >= num_frames:
                print(f" Collected {collected} frames for '{label}' ({mode})")
                break
        else:
            if detected:
                cv2.putText(frame, f"{mode.capitalize()} detected. Press 'c' to start collecting.",
                            (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, f"No {mode} detected",
                            (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.imshow("Data Collection", frame)
        key = cv2.waitKey(10) 

        if key == ord('c') and not collecting and detected:
            collecting = True
            print(f"Started collecting data for '{label}' ({mode})")
            time.sleep(0.5)  # tiny delay before start
        elif key == ord('q'):
            print("Quitting data collection.")
            collected = 0
            break

    cap.release()
    cv2.destroyAllWindows()

    if data:
        df = pd.DataFrame(data)
        csv_path = os.path.join(save_dir, f"{label}.csv")
        if os.path.exists(csv_path):
            df.to_csv(csv_path, mode='a', index=False, header=False)
        else:
            df.to_csv(csv_path, index=False)
        print(f"Saved {len(data)} frames to {csv_path}")
    else:
        print("No data collected.")

def main():
    people = int(input("Enter number of people to collect data for: "))
    face_labels = ['happy', 'sad', 'angry']
    hand_labels = ['thumbs_up', 'peace', 'pointing', 'open_hand']

    for p in range(people):
        print(f"\n=== Collecting data for person {p+1}/{people} ===")

        for label in face_labels:
            collect_label_data(label, 'face')

        for label in hand_labels:
            collect_label_data(label, 'hand')

if __name__ == "__main__":
    main()


Enter number of people to collect data for:  1



=== Collecting data for person 1/1 ===

Prepare to collect 'happy' data for face. Press 'c' to start collecting 200 frames.
Press 'q' to quit anytime.
Quitting data collection.
No data collected.

Prepare to collect 'sad' data for face. Press 'c' to start collecting 200 frames.
Press 'q' to quit anytime.
Quitting data collection.
No data collected.
